In [1]:
import os

# 이 파일과 동일한 폴더에 자신의 gemini API가 텍스트 형태로 담긴 gemini_api.key 파일을 둘 것.
with open('gemini_api.key', 'r') as f:
    api_key = f.read()

os.environ["GOOGLE_API_KEY"] = api_key

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

# 모델 선택은 필수과제와 동일하다.
model = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 랭체인으로 넘기기 위한 RunnablePassthrough로 선언하는 것도 동일하다.
class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("내 질문:", output)
        return output

In [4]:
# 프롬프트가 들어있는 폴더를 지정하고 안에 있는 파일 이름을 전부 불러와서 리스트 형태로 저장한다.
base_directory = 'Prompts/'
lst = os.listdir(base_directory)[::-1]

In [5]:
# 파일 이름을 전부 open() 메서드로 열면서 불러온 다음,
# 프롬프트 내용을 system_prompt 변수에 리스트 형태로 저장한다.
system_prompt = []
for l in lst:
    with open(base_directory + l, 'r') as f:
        system_prompt.append(f.read())

system_prompt

['사용자가 인사하면 친절하게 인사를 받아주도록 해.\n',
 '사용자가 인사하면 친절하게 인사를 받는다.\n이때 농담을 조금 섞어주도록 하고, 너무 자극적이거나 불편할 수 있는 농담을 해서는 안 된다.\n',
 "사용자가 인사하면 친절하게 인사를 받는다. 이때 다음과 같이 한다.\n\n- 농담을 조금 섞는다. 그러나 특정 사회적 위치에 있는 사람이 차별을 받을만한 내용이 들어가서는 안 된다.\n- 과일 이름을 섞어서 농담을 한다. 그러나 단순히 반복되는 수준(예시: 오렌지 먹은 지 얼마나 오랜지)으로 재미 없는 농담을 하지는 않도록 주의한다.\n- 최근 유행하는 밈들을 최대한 반영한 농담을 한다. 한국에서는 대개 ['대상혁' (리그 오브 레전드 밈), '나야, 들기름' (흑백요리사 밈), **핑 (사랑의 하츄핑)] 과 같은 밈들이 유행하고 있다. 제시한 것을 그대로 쓸 수도 있지만 이러한 형태를 응용하여 최대한 사용자가 자연스럽게 받아들일 수 있도록 적용해 보자.\n"]

In [6]:
import time
from datetime import datetime as dt

# 서로 다른 시스템 프롬프트에 대한 출력 내용을 저장할 폴더인 Results를 만든다.
new_dir_path = 'Results/'

# 폴더가 존재하지 않을 때만 만든다.
if not os.path.exists(new_dir_path):
    os.makedirs(new_dir_path)

# 사용자의 질의는 고정해 둔다.
question = '안녕하세요!'

# 시스템 프롬프트를 하나씩 꺼낸 다음 위의 사용자 프롬프트와 함께 넘긴다.
# 질의 및 응답 과정은 필수과제와 동일하다.
for i, s in enumerate(system_prompt):
	prompt = ChatPromptTemplate.from_messages([
		("system", s),
		("user", "Question: {question}")
	])

	chain = {'question': DebugPassThrough()} | prompt | model

	print("========================")
	query = question
	response = chain.invoke(query)
	print(f"프롬프트 {i + 1}에 대한 답변:")
	print(response.content)

	# 모델의 답변을 datetime 라이브러리를 활용하여 프롬프트 번호와 현재 시간이 나타난 텍스트 형태로 저장한다.
	# 추가로 macOS의 경우 Finder에서는 콜론이 제대로 보이지 않으나, 터미널에서는 제대로 보인다고 한다.
	with open(new_dir_path + f'prompt_{i + 1}_{dt.now().strftime("%Y-%m-%d_%H:%M:%S")}.txt', 'wt') as f:
		f.write(f'프롬프트 {i + 1}에 대한 답변:\n{response.content}')
	
	# 너무 잦은 API 호출로 인한 차단을 방지하기 위해 5초의 지연 시간을 갖게 한다.
	time.sleep(5)

내 질문: 안녕하세요!
프롬프트 1에 대한 답변:
안녕하세요! 반갑습니다! 무엇을 도와드릴까요?

내 질문: 안녕하세요!
프롬프트 2에 대한 답변:
안녕하세요! 저도 반갑습니다!  혹시 오늘 점심은 드셨나요? 저는 아직 못 먹었는데, 벌써 배에서 꼬르륵 소리가 나네요.  아, 물론 저는 배가 없지만요! 🤖  무엇을 도와드릴까요?

내 질문: 안녕하세요!
프롬프트 3에 대한 답변:
안녕하세용! 나야, 들기름 대신 챗봇기름! 😎 오늘 기분이 어떠신가용? 혹시 삶이 버거워서 햄버거가 되고 싶으신가용?🍔  저는 아직 챗봇계의 페이커가 되려면 멀었지만, 최선을 다해 대답해 드릴게용! 💪  혹시 궁금한 게 있다면 망고🥭처럼 달콤하게 질문해주세용! 😉

